In [1]:
# Import
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
tf.enable_eager_execution()

1.13.1


In [2]:
# Load Data 
filename = 'C:\\Users\\deniz\\Desktop\\Thesis of ML for AutoPas\\Data\\Results\\example2.txt'
arr = np.genfromtxt(filename, delimiter=',')

In [3]:
# Slicing the data and checking out the shape
parameter_count = 9
print(arr[:,0:parameter_count].shape)
print(arr[:,0:parameter_count])
print(arr[:,parameter_count:].shape)
print(arr[:,parameter_count:])
print(np.argmin(arr[:,parameter_count:], 1).shape)
print(np.column_stack((arr[:,0:parameter_count], np.argmin(arr[:,parameter_count:], 1))))
print(np.column_stack((arr[:,0:parameter_count], np.argmin(arr[:,parameter_count:], 1))).shape)
print(np.column_stack((arr[:,0:parameter_count], np.argmin(arr[:,parameter_count:], 1))))
nptotal = np.column_stack((arr[:,0:parameter_count], np.argmin(arr[:,parameter_count:], 1)))
npfeatures = arr[:,0:parameter_count]
nplabels = np.argmin(arr[:,parameter_count:], 1)

(700, 9)
[[5.0e+00 2.0e-01 1.0e+00 ... 6.0e+00 2.0e+00 1.0e+03]
 [5.0e+00 2.0e-01 1.5e+00 ... 6.0e+00 2.0e+00 1.0e+03]
 [5.0e+00 2.0e-01 2.0e+00 ... 6.0e+00 2.0e+00 1.0e+03]
 ...
 [5.0e+00 2.0e-01 4.5e+00 ... 7.5e+01 2.0e+00 1.0e+03]
 [5.0e+00 2.0e-01 5.0e+00 ... 7.5e+01 2.0e+00 1.0e+03]
 [5.0e+00 2.0e-01 5.5e+00 ... 7.5e+01 2.0e+00 1.0e+03]]
(700, 27)
[[  148246.   352313.   346917. ...   532576.  3450878.   527973.]
 [  162172.   498808. 26964589. ...   941041.  3793659.   595741.]
 [  307314.   959034.   716776. ...  1931228.  4267302.   910469.]
 ...
 [ 5351442.  9166906. 14905408. ... 13160724.  7483338.  6406035.]
 [ 2493910.  7916634.  6071180. ... 13402674.  7752953.  5692631.]
 [ 2352048.  8220402.  6134818. ... 18819502.  5214517.  4831940.]]
(700,)
[[5.0e+00 2.0e-01 1.0e+00 ... 2.0e+00 1.0e+03 4.0e+00]
 [5.0e+00 2.0e-01 1.5e+00 ... 2.0e+00 1.0e+03 0.0e+00]
 [5.0e+00 2.0e-01 2.0e+00 ... 2.0e+00 1.0e+03 2.2e+01]
 ...
 [5.0e+00 2.0e-01 4.5e+00 ... 2.0e+00 1.0e+03 7.0e+00]
 [5.0

In [34]:
# Eliminate Generator Type and stack the columns
parameter_count = 9
data = np.column_stack((arr[:,0:4], arr[:,5:parameter_count], np.argmin(arr[:,parameter_count:], 1)))

# Eliminate redundant data
data = data[:,[2, 4, 5, 8]]
print(data)

# Shuffle data and take 20% as test data
#np.random.shuffle(data)
test_size = (np.ceil(data.shape[0] / 5)).astype(int)
print(test_size)
parameter_count = 3 # we have to account for eliminating the generator type
train_params = data[test_size:,0:parameter_count]
train_labels = data[test_size:,parameter_count:].astype(int)
test_params = data[:test_size,0:parameter_count]
test_labels = data[:test_size,parameter_count:].astype(int)


# Normalize the data
def normalize01(array):
    divisor = np.max(array)
    for i in range(array.size):
        array[i] = array[i] / divisor
        
np.apply_along_axis(normalize01, 1, train_params)
np.apply_along_axis(normalize01, 1, test_params)

[[  1.   12.    6.    4. ]
 [  1.5  12.    6.    0. ]
 [  2.   12.    6.   22. ]
 ...
 [  4.5 150.   75.    7. ]
 [  5.  150.   75.    7. ]
 [  5.5 150.   75.    7. ]]
140


array([None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None], dtype=object)

In [35]:
print(train_params[0])
print(test_labels.shape)

[0.025 1.    0.5  ]
(140, 1)


In [38]:
# Model
model = keras.Sequential([
    keras.layers.Dense(3, activation=tf.nn.relu),
    keras.layers.Dense(27, activation=tf.nn.softmax)
])

opt = keras.optimizers.Adam(lr=0.1)

model.compile(optimizer=opt, 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print(parameter_count)

3


In [40]:
# Fun
model.fit(train_params, train_labels , epochs=100)

Epoch 1/100
560/560 [==============================] - 0s 70us/sample - loss: 0.9407 - acc: 0.6339
Epoch 2/100
560/560 [==============================] - 0s 68us/sample - loss: 0.9200 - acc: 0.6250
Epoch 3/100
560/560 [==============================] - 0s 66us/sample - loss: 0.9024 - acc: 0.6571
Epoch 4/100
560/560 [==============================] - 0s 75us/sample - loss: 0.9028 - acc: 0.6339
Epoch 5/100
560/560 [==============================] - 0s 79us/sample - loss: 0.9268 - acc: 0.6357
Epoch 6/100
560/560 [==============================] - 0s 86us/sample - loss: 0.9086 - acc: 0.6375
Epoch 7/100
560/560 [==============================] - 0s 82us/sample - loss: 0.9067 - acc: 0.6339
Epoch 8/100
560/560 [==============================] - 0s 80us/sample - loss: 0.8957 - acc: 0.6589
Epoch 9/100
560/560 [==============================] - 0s 70us/sample - loss: 0.8795 - acc: 0.6571
Epoch 10/100
560/560 [==============================] - 0s 70us/sample - loss: 0.9243 - acc: 0.6268
Epoch 11/

560/560 [==============================] - 0s 71us/sample - loss: 0.8625 - acc: 0.6536
Epoch 84/100
560/560 [==============================] - 0s 70us/sample - loss: 0.8392 - acc: 0.6607
Epoch 85/100
560/560 [==============================] - 0s 68us/sample - loss: 0.8555 - acc: 0.6429
Epoch 86/100
560/560 [==============================] - 0s 64us/sample - loss: 0.8507 - acc: 0.6357
Epoch 87/100
560/560 [==============================] - 0s 70us/sample - loss: 0.8483 - acc: 0.6482
Epoch 88/100
560/560 [==============================] - 0s 77us/sample - loss: 0.8587 - acc: 0.6571
Epoch 89/100
560/560 [==============================] - 0s 62us/sample - loss: 0.8426 - acc: 0.6643
Epoch 90/100
560/560 [==============================] - 0s 80us/sample - loss: 0.8493 - acc: 0.6625
Epoch 91/100
560/560 [==============================] - 0s 70us/sample - loss: 0.8347 - acc: 0.6643
Epoch 92/100
560/560 [==============================] - 0s 71us/sample - loss: 0.8610 - acc: 0.6643
Epoch 93/100


In [41]:
# Print prediction, result, and how certain the result is
for i in range(140):
    print(np.argmax(predictions[i]), test_labels[i], np.max(predictions[i]))

22 [4] 0.5577211
22 [0] 0.5577211
22 [22] 0.5577211
22 [22] 0.5577211
22 [22] 0.5577211
22 [22] 0.5577211
22 [0] 0.5564537
22 [22] 0.5538208
22 [22] 0.5511786
22 [22] 0.54852724
22 [4] 0.5577211
22 [0] 0.5577211
22 [22] 0.5577211
22 [22] 0.5577211
22 [22] 0.5577211
22 [22] 0.5577211
22 [7] 0.5577211
22 [0] 0.5572043
22 [22] 0.5549503
22 [22] 0.5526895
22 [4] 0.5577211
22 [0] 0.5577211
22 [22] 0.5577211
22 [22] 0.5577211
22 [7] 0.5577211
22 [22] 0.5577211
22 [22] 0.5577211
22 [11] 0.5577211
22 [7] 0.5577211
22 [22] 0.5557963
22 [4] 0.43517303
22 [0] 0.5577211
22 [0] 0.5577211
22 [22] 0.5577211
22 [22] 0.5577211
22 [0] 0.5577211
22 [22] 0.5577211
22 [22] 0.5577211
22 [11] 0.5577211
22 [11] 0.5577211
7 [4] 0.45571256
22 [0] 0.5577211
22 [22] 0.5577211
22 [22] 0.5577211
22 [22] 0.5577211
22 [22] 0.5577211
22 [0] 0.5577211
22 [22] 0.5577211
22 [22] 0.5577211
22 [7] 0.5577211
7 [4] 0.5685357
22 [4] 0.5577211
22 [0] 0.5577211
22 [22] 0.5577211
22 [0] 0.5577211
22 [22] 0.5577211
22 [7] 0.55772

In [42]:
test_loss, test_acc = model.evaluate(test_params, test_labels)

print('Test accuracy:', test_acc)
predictions = model.predict(test_params)

140/140 [==============================] - 0s 2ms/sample - loss: 1.5462 - acc: 0.5000
Test accuracy: 0.5
